```python
# Copyright 2022 Bloomberg Finance L.P.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
```

# Portfolio Learning Item

### What is a portfolio

We'll now learn about the final representation and class in our portfolio manager, the portfolio. We've discussed how individuals or institutions can have multiple accounts for different purposes. A portfolio is a collection of these accounts, but also is any collection of investments/financial assets, including stock, bonds, derivatives, ETFs, etc. Accounts are technically a portfolio, but large firms will usually have multiple accounts with different trading strategies. The collection of these accounts represents their portfolio.

For example, I could have account *Tech Strategy* which contains the positions:

***2,000*** shares of ***MSFT USD***,
***200*** shares of ***AAPL USD***, and 
***100*** shares of ***TSLA USD***

And an account *Healthcare Strategy* which contains the positions:

***1,000*** shares of ***UNH USD***,
***200*** shares of ***HCA USD***, and 
***100*** shares of ***MSFT USD***

My entire portfolio is the union of these two accounts. It's important to note that accounts within a portfolio don't need to be mutually exclusive in their positions.

### Problem Definition

We want to build a class that allows for the management of a portfolio. We should be able to construct a portfolio using a set of accounts and a portfolio name. We should be able to query the portfolio object retrieving the name of the portfolio, a subset of accounts based on a set of account names or return a collection of all accounts. We should also be able to remove/add accounts based on a set of account names or account objects respectively. Removals should ignore non-existent names & additions can replace existing accounts.

- Allow for a portfolio to be created with a set of accounts and a portfolio name
- Allow for querying of the current portfolio's name
- Allow for querying of all accounts within the portfolio
- Allow for the querying for accounts in the portfolio that match the filter criteria, returning an iterable
    - The user can give a set of account names. Any accounts that match these names should be returned
    - The user can give a set of security names or objects. Any accounts that contain one or more positions from the set should be returned
    - The user can combine both queries creating a dual filter. Accounts that pass both filters should be returned.
- Allow for accounts to be added to the portfolio with a set of account objects. Incoming accounts can replace existing accounts
- Allow for the removal of accounts from the portfolio with a set of account names. Names not in the portfolio should be ignored.

### Provided Tools

#### *Data Source*

For this section no data generators are provided

#### *Solution Interface*

Your solution will need to follow the interface provided in the lab. Below is a snippet of the interface for securities that you can inherit from. The methods that will be tested are displayed & will need to be overwritten with your implementation. You're free to add more methods then what is displayed in the interface!

```python
#filename interfaces.portfolioInterface.py
#Portfolio Class Interface

from typing import Set, Iterable
from .accountInterface import accountInterface
from .securityInterface import securityInterface
class portfolioInterface():
    def __init__(self, portfolioName: str, accounts: Set[accountInterface]) -> None:
        pass

    def getAllAccounts(self) -> Iterable[accountInterface]:
        pass

    def getAccounts(self, accountNamesFilter:Set[str], securitiesFilter:Set) -> Iterable[accountInterface]:
        pass 

    def addAccounts(self, accounts: Set[accountInterface]) -> None:
        pass

    def removeAccounts(self, accountNames: Set[str]) -> None:
        pass

```

#### *Testing*

Once you have completed & saved your solution you can run the test file to validate that your solution works as expected. For the test to run the following need to be true.
- Saved code to file **implementations/portfolioSolution.py**
- Create a class with the name **portfolio** that inherits from **portfolioInterface**

### Stretch Goals

If you complete your class & have a solution with valid tests try completing the following stretch goals 

- Update your class init to handle the account set being optional
- Add custom implementation for class's __str__ method.
- Add a method to remove clear the entire portfolio of all accounts
- Develop tests for the new methods created

In [ ]:
#%%writefile ../implementations/portfolioSolution.py 
#Uncomment line above & run cell to save solution
#TODO Define class that implements portFolioInterface & allows for the management of a portfolio

In [ ]:
#%conda install ipytest
#%pip install ipytest
import ipytest

ipytest.autoconfig()

In [ ]:
%%ipytest -qq
from implementations.accountSolution import account
from implementations.positionSolution import position
import implementations.portfolioSolution
import pytest
import importlib
importlib.reload(implementations.portfolioSolution)

def test_GetAllAccounts():
    #GIVEN
    PORTFOLIO_NAME = "TestPortfolio"
    ACCOUNT_A_POSITIONS = [
        position("MSFT US Equity", 1000),
        position("TSLA US Equity", 2000),
    ]
    ACCOUNT_B_POSITIONS = [
        position("APPL US Equity", 500),
        position("RIVN US Equity", 1000),
    ]
    ACCOUNT_A = account(ACCOUNT_A_POSITIONS, "Account A")
    ACCOUNT_B = account(ACCOUNT_B_POSITIONS, "Account B")

    ACCOUNTS = set([ACCOUNT_A, ACCOUNT_B])
    EXPECTED_ACCOUNTS = {acc.getName():True for acc in ACCOUNTS}

    #WHEN
    p = implementations.portfolioSolution.portfolio(PORTFOLIO_NAME, ACCOUNTS)

    #EXPECT
    ALL_ACCS = p.getAllAccounts()
    
    for acc in ALL_ACCS:
        assert acc.getName() in EXPECTED_ACCOUNTS
        EXPECTED_ACCOUNTS[acc.getName()] = False
    
    assert True not in EXPECTED_ACCOUNTS

@pytest.mark.parametrize("inputAccount, inputSecurity, expectedMap", (
    ([], [], {"Account A": True, "Account B": True, "Account C": True}),
    (["Account A", "Account B", "Account DNE"], [], {"Account A": True, "Account B": True}),
    ([], ["IBM US Equity", "FOOD US Equity"], {"Account A": True, "Account B": True, "Account C": True}),
    (["Account B", "Account C"], ["IBM US Equity"], {"Account B": True})
))
def test_GetSubsetAccounts(inputAccount, inputSecurity, expectedMap):
    #GIVEN
    PORTFOLIO_NAME = "TestPortfolio"
    ACCOUNT_A_POSITIONS = [
        position("MSFT US Equity", 1000),
        position("TSLA US Equity", 2000),
        position("IBM US Equity", 3000)
    ]
    ACCOUNT_B_POSITIONS = [
        position("APPL US Equity", 500),
        position("RIVN US Equity", 1000),
        position("IBM US Equity", 1234)
    ]
    ACCOUNT_C_POSITIONS = [
        position("SWS US Equity", 241),
        position("CORE US Equity", 4213),
        position("FOOD US Equity", 1234)
    ]
    ACCOUNT_A = account(ACCOUNT_A_POSITIONS, "Account A")
    ACCOUNT_B = account(ACCOUNT_B_POSITIONS, "Account B")
    ACCOUNT_C = account(ACCOUNT_C_POSITIONS, "Account C")

    ACCOUNTS = set([ACCOUNT_A, ACCOUNT_B, ACCOUNT_C])

    #WHEN
    p = implementations.portfolioSolution.portfolio(PORTFOLIO_NAME, ACCOUNTS)

    #EXPECT
    FILTERED_ACCOUNTS = p.getAccounts(inputAccount, inputSecurity)
    
    for acc in FILTERED_ACCOUNTS:
        assert acc.getName() in expectedMap
        expectedMap[acc.getName()] = False
    
    assert True not in expectedMap

def test_AddAccountsNoOverwrite():
    #GIVEN
    PORTFOLIO_NAME = "TestPortfolio"
    ACCOUNT_A_POSITIONS = [
        position("MSFT US Equity", 1000),
        position("TSLA US Equity", 2000),
    ]
    ACCOUNT_B_POSITIONS = [
        position("APPL US Equity", 500),
        position("RIVN US Equity", 1000),
    ]
    ACCOUNT_A = account(ACCOUNT_A_POSITIONS, "Account A")
    ACCOUNT_B = account(ACCOUNT_B_POSITIONS, "Account B")

    ACCOUNTS = set([ACCOUNT_A, ACCOUNT_B])
    EXPECTED_ACCOUNTS = {acc.getName():True for acc in ACCOUNTS}

    #WHEN
    p = implementations.portfolioSolution.portfolio(PORTFOLIO_NAME, [])
    p.addAccounts(ACCOUNTS)

    #EXPECT
    ALL_ACCS = p.getAllAccounts()
    
    for acc in ALL_ACCS:
        assert acc.getName() in EXPECTED_ACCOUNTS
        EXPECTED_ACCOUNTS[acc.getName()] = False
    
    assert True not in EXPECTED_ACCOUNTS

def test_AddAccountOverwrite():
    #GIVEN
    PORTFOLIO_NAME = "TestPortfolio"
    ACCOUNT_A_POSITIONS = [
        position("MSFT US Equity", 1000),
        position("TSLA US Equity", 2000),
    ]
    ACCOUNT_B_POSITIONS = [
        position("APPL US Equity", 500),
        position("RIVN US Equity", 1000),
    ]
    ACCOUNT_A = account(ACCOUNT_A_POSITIONS, "Account A")
    ACCOUNT_B = account(ACCOUNT_B_POSITIONS, "Account B")
    ACCOUNTS = set([ACCOUNT_A, ACCOUNT_B])

    #WHEN
    p = implementations.portfolioSolution.portfolio(PORTFOLIO_NAME, ACCOUNTS)
    ACCOUNT_B_POSITIONS_NEW = [
        position("PELO US Equity", 500),
        position("IBM US Equity", 1000),
    ]
    ACCOUNT_B_NEW = account(ACCOUNT_B_POSITIONS_NEW, "Account B")
    EXPECTED_ACCOUNTS = {
        "Account A": ACCOUNT_A_POSITIONS,
        "Account B": ACCOUNT_B_POSITIONS_NEW
    }
    p.addAccounts([ACCOUNT_B_NEW])

    #EXPECT
    ALL_ACCS = p.getAllAccounts()
    
    for acc in ALL_ACCS:
        assert acc.getName() in EXPECTED_ACCOUNTS
        POS_EXPECTED = {x.getSecurity().getName(): x.getPosition() for x in EXPECTED_ACCOUNTS[acc.getName()]}
        RETURN_POS = acc.getAllPositions()

        for pos in RETURN_POS:
            assert pos.getSecurity().getName() in POS_EXPECTED
            assert POS_EXPECTED[pos.getSecurity().getName()] == pos.getPosition()

            #Remove the validate position from out expected map
            del POS_EXPECTED[pos.getSecurity().getName()]

        assert len(POS_EXPECTED) == 0

def test_RemoveAccounts():
    #GIVEN
    PORTFOLIO_NAME = "TestPortfolio"
    ACCOUNT_A_POSITIONS = [
        position("MSFT US Equity", 1000),
        position("TSLA US Equity", 2000),
    ]
    ACCOUNT_B_POSITIONS = [
        position("APPL US Equity", 500),
        position("RIVN US Equity", 1000),
    ]
    ACCOUNT_A = account(ACCOUNT_A_POSITIONS, "Account A")
    ACCOUNT_B = account(ACCOUNT_B_POSITIONS, "Account B")

    ACCOUNTS = set([ACCOUNT_A, ACCOUNT_B])
    EXPECTED_ACCOUNTS = {"Account A": True}

    #WHEN
    p = implementations.portfolioSolution.portfolio(PORTFOLIO_NAME, ACCOUNTS)
    p.removeAccounts(["Account B", "Account DNE"])

    #EXPECT
    ALL_ACCS = p.getAllAccounts()
    
    for acc in ALL_ACCS:
        assert acc.getName() in EXPECTED_ACCOUNTS
        EXPECTED_ACCOUNTS[acc.getName()] = False
    
    assert True not in EXPECTED_ACCOUNTS